In [ ]:
from IPython import get_ipython
from IPython.display import display

In [ ]:
!pip install tensorflow==2.12.0
!pip install scikeras
!pip install scikit-optimize

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from scikeras.wrappers import KerasClassifier

In [ ]:
! mkdir ~/.kaggle

In [ ]:
!cp /content/drive/MyDrive/CollabData/kaggle_API/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download cakrulgaming/indonesia-traffic-sign

In [ ]:
! unzip indonesia-traffic-sign.zip

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    'test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

In [ ]:
def create_model(learning_rate=0.001, dropout_rate=0.5):
    base_model = ResNet50(weights='imagenet', include_top=False)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x) # Feature engineering
    x = tf.keras.layers.Dropout(dropout_rate)(x) # Regularization
    predictions = Dense(num_classes, activation='softmax')(x) # num_classes is the number of traffic sign classes
    model = Model(inputs=base_model.input, outputs=predictions)

    # Fine-tuning: Unfreeze some layers for training
    for layer in base_model.layers[-10:]:
        layer.trainable = True

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
model = KerasClassifier(model=create_model, verbose=0)
search_space = {
    'learning_rate': Real(1e-5, 1e-2, prior='log-uniform'),
    'dropout_rate': Real(0.2, 0.8),
    'batch_size': Integer(16, 64)
}

opt = BayesSearchCV(
    model,
    search_space,
    n_iter=32, # Number of iterations
    cv=3, # Number of cross-validation folds
    n_jobs=-1 # Use all available cores
)

opt.fit(train_generator)

In [ ]:
best_model = opt.best_estimator_
history = best_model.fit(train_generator, epochs=10)

In [ ]:
y_pred = best_model.predict(test_generator)
y_true = test_generator.classes

#Calculate metrics
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')
cm = confusion_matrix(y_true, y_pred)
auc = roc_auc_score(y_true, y_pred, multi_class='ovr')

#Print metrics - skipped to avoid execution
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")
print(f"Confusion Matrix:\n{cm}")
print(f"AUC: {auc}")

In [ ]:
class_weights = compute_class_weight('balanced', classes=np.unique(train_generator.classes), y=train_generator.classes)
best_model.fit(train_generator, epochs=10, class_weight=class_weights)